In [1]:
from torchPIV import OfflinePIV
import torch

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

import cv2
import os
import gc

In [2]:
gpu_enabled = torch.cuda.is_available()
gpu_enabled = False

if gpu_enabled:
    device = torch.cuda.get_device_name()
else:
    device = 'cpu'
    
print("Current device:", device)

Current device: cpu


In [3]:
for f in os.listdir('test'):
    os.remove(f'test/{f}')

test_arr = np.random.choice([0, 255], size=(128, 128), p=[19/20, 1/20]).astype(np.uint8)
cv2.imshow('', test_arr)
cv2.waitKey(1000)
cv2.destroyAllWindows()

for i in range(50):
    cv2.imwrite(f'test/{i}.jpg', test_arr)
    test_arr = np.roll(test_arr, 3)

In [5]:
# folder, file_format, (mode, dt) = r"test images", "bmp", ("pairs", 12)
# folder, file_format, (mode, dt) = r"plume simulation", "jpg", ("sequential", 500)
folder, file_format, (mode, dt) = r"test", "jpg", ("sequential", 500)

In [121]:
torch.cuda.empty_cache()
gc.collect()

0

In [9]:
piv_gen = OfflinePIV(
    folder=folder, # Path to experiment
    device=device, # Device name
    file_fmt=file_format,
    wind_size=32,
    overlap=16,
    dt=dt, # Time between frames, mcs
    scale = 0.02, # mm/pix
    multipass=2,
    multipass_mode="CWS", # CWS or DWS
    multipass_scale=2.0, # Window downscale on each pass
    folder_mode=mode # Pairs or sequential frames 
)

results = []
for out in piv_gen():
    x, y, vx, vy = out
    results.append(out)

Load time 0.001 sec Iteration finished in 0.031 sec Warning! to many false vectors
Warning! to many false vectors
[[0.11997068 0.11994033 0.12031644 0.120128   0.12018423 0.11994585
  0.11995454 0.12003793 0.12008741 0.12006717 0.11987569 0.11988948
  0.11976151 0.12013743 0.1200713 ]
 [0.11994365 0.11989571 0.12013733 0.12008069 0.12025101 0.11992341
  0.11995713 0.12006474 0.12002539 0.11988115 0.11979104 0.12011303
  0.11974257 0.12006124 0.120069  ]
 [0.12004701 0.12008028 0.12009803 0.11994347 0.11990527 0.11977132
  0.11990907 0.12014709 0.12007028 0.11990957 0.11973768 0.12025324
  0.11944856 0.11969312 0.11959176]
 [0.12030333 0.12012289 0.12007331 0.12001742 0.11996469 0.11994622
  0.12005547 0.120038   0.12009358 0.12007284 0.12010448 0.1199852
  0.11982905 0.12000388 0.12000342]
 [0.12058394 0.12037484 0.12005971 0.12007474 0.120166   0.12044487
  0.12025984 0.12008193 0.12004772 0.12011377 0.12001441 0.11983481
  0.12009636 0.1210638  0.12087298]
 [0.12080289 0.12108464 0.1

In [10]:
files = os.listdir(folder)

velocities = []

for _, _, vx, vy in results:
    print(vx)
    abs_velocity = np.sqrt(vx ** 2 + vy ** 2)
    velocities.append(abs_velocity)
    
velocities = np.array(velocities)
velocities /= np.max(velocities)

for idx, (x, y, vx, vy) in enumerate(results):
    img = cv2.imread(folder + f"/{files[idx]}", cv2.IMREAD_GRAYSCALE)

    new_x = x/piv_gen._scale
    new_y = y/piv_gen._scale

    abs_velocity = np.sqrt(vx ** 2 + vy ** 2)
    abs_velocity /= np.max(abs_velocity)

    plt.imshow(img, cmap='gray')
    plt.quiver(new_x, new_y, vx, vy, velocities[idx], cmap='jet')
    plt.show()

[[0.11997068 0.11994033 0.12031644 0.120128   0.12018423 0.11994585
  0.11995454 0.12003793 0.12008741 0.12006717 0.11987569 0.11988948
  0.11976151 0.12013743 0.1200713 ]
 [0.11994365 0.11989571 0.12013733 0.12008069 0.12025101 0.11992341
  0.11995713 0.12006474 0.12002539 0.11988115 0.11979104 0.12011303
  0.11974257 0.12006124 0.120069  ]
 [0.12004701 0.12008028 0.12009803 0.11994347 0.11990527 0.11977132
  0.11990907 0.12014709 0.12007028 0.11990957 0.11973768 0.12025324
  0.11944856 0.11969312 0.11959176]
 [0.12030333 0.12012289 0.12007331 0.12001742 0.11996469 0.11994622
  0.12005547 0.120038   0.12009358 0.12007284 0.12010448 0.1199852
  0.11982905 0.12000388 0.12000342]
 [0.12058394 0.12037484 0.12005971 0.12007474 0.120166   0.12044487
  0.12025984 0.12008193 0.12004772 0.12011377 0.12001441 0.11983481
  0.12009636 0.1210638  0.12087298]
 [0.12080289 0.12108464 0.12001749 0.11988383 0.12032196 0.12084011
  0.12031474 0.11995498 0.12001312 0.12000066 0.11987362 0.11987393
  0.1

In [6]:
for img_file in os.listdir(folder):
    img = cv2.imread(folder + f"/{img_file}", cv2.IMREAD_GRAYSCALE)
    xsize, ysize = img.shape
    
    newx, newy = int(xsize), int(ysize)
    img = cv2.resize(img, (newx, newy), interpolation= cv2.INTER_LINEAR)
    
    cv2.imshow(str(img_file), img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()